In [5]:
!python --version

Python 3.7.6


In [ ]:
!pwd

In [ ]:
CONFIG = 'basic_rnn'
BUNDLE_PATH = '/root/magenta/magenta/models/melody_rnn/mag/basic_rnn.mag'
OUTPUT_DIR = '/root/magenta/magenta/models/melody_rnn/output/2020.3.7-01'

# 用 pre-trained model 生成序列

In [ ]:
!melody_rnn_generate \
--config='attention_rnn' \
--bundle_file='/root/magenta/magenta/models/melody_rnn/mag/attention_rnn.mag' \
--output_dir='/root/magenta/magenta/models/melody_rnn/output/2020.3.30' \
--num_outputs=5 \
--num_steps=256 \
--primer_midi='/root/magenta/magenta/models/melody_rnn/primer.mid'

# 制作数据集

In [6]:
INPUT_DIRECTORY='/download/clean_midi'

# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD=/root/notesequences.tfrecord

convert_dir_to_note_sequences \
  --input_dir=$INPUT_DIRECTORY \
  --output_file=$SEQUENCES_TFRECORD \
  --recursive

SyntaxError: invalid syntax (<ipython-input-6-bc978e391011>, line 1)

# 训练模型

## Create SequenceExamples

In [1]:
!melody_rnn_create_dataset \
--config=attention_rnn \
--input=/root/notesequences.tfrecord \
--output_dir=/download/melody_rnn/sequence_examples \
--eval_ratio=0.10

SyntaxError: invalid syntax (<ipython-input-1-3747404f7367>, line 3)

# 保存模型

In [ ]:
from magenta.models.shared.sequence_generator import BaseSequenceGenerator

In [ ]:
!melody_rnn_generate \
--config=attention_rnn \
--run_dir=/root/magenta/magenta/models/melody_rnn/logdir/run1 \
--hparams="batch_size=64,rnn_layer_sizes=[64,64]" \
--bundle_file=/root/magenta/magenta/models/melody_rnn/mag/attention_rnn.mag \
--save_generator_bundle